In [1]:
import torch

In [2]:
input_ids = torch.tensor([2, 3, 5, 1])

In [3]:
vocab_size = 6
output_dim = 3

In [4]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


将嵌入层应用于token ID

In [5]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


将嵌入层应用于所有四个输入ID 

In [7]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [8]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # 编码整个文本
        # 循环遍历数据，使用滑动窗口将数据分成重叠的最大长度序列
        # 假设token_ids的长度大于max_length
        for i in range(0,len(token_ids) - max_length, stride):
            # 输入的区域是i到i+maxlength，左闭右开，因此最后一个token不参与输入
            input_chunk = token_ids[i:i + max_length]
            # 第一个token不作为预测目标
            target_chunk = token_ids[i+1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    # 返回数据集的行数
    def __len__(self):
        return len(self.input_ids)
    # 返回数据集中单独一行数据
    def __getitem__(self, idx):
        return self.input_ids[idx],self.target_ids[idx]
import tiktoken
def create_dataloader_v1(txt, 
                         batch_size=4, 
                         max_length=256, 
                         stride=8, 
                         shuffle=True, 
                         drop_last=True, 
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2") # 初始化tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) # 创建数据集
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last, # drop_last=True时，如果最后一个批次小于指定的 batch_size，则丢弃它，以防止训练期间出现损失峰值。
        num_workers=num_workers # 用于预处理的 CPU 进程数
    )

    return dataloader

with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [11]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
   stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


将tokenIDs嵌入到256维向量中

In [12]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


实现绝对位置嵌入

In [13]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [14]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
